# Socket - Low-Level Networking Interface  

## 설정  
### 소캣 생성  
socket(family, type)  

- family  
| 이름 | 프로토콜 체계(Protocol Family) |  
| ------ | ------ |  
| PF_INET | IPv4 인터넷 프로토콜 체계 |  
| PF_INET6 | IPv6 인터넷 프로토콜 체계 |  
| PF_LOCAL | 로컬 통신을 위한 UNIX 프로토콜 체계 |  
| PF_PACKET | LOW Level 소캣을 위한 프로토콜 체계 |  
| PF_IPX | IPX 노벨 프로토콜 체계 |  

- type  
1. SOCK_STREAM(연결지향형 소켓)  

2. SOCK_DGRAM(비연결지향형 소켓)  


### 소캣 옵션 설정  
setsockopt(socket, level, optname, optval, optlen)
- socket:
- level:
- optname:
- optval:
- optlen:

### IP/PORT 할당  
bind(host, port)  

### 연결 대기  
listen()  

### EGIO


## Socket Families  

## Module contents  

### 1. 예외(Exceptions)  

### 2. 상수(Constants)  

### 3. 함수(Functions)  
- 소켓만들기

socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)

- EGIO 설정  
socket.socket(  
    family=AF_INET  
    , type=SOCK_DGRAM  
    , proto=IPPROTO_UDP  
)  

### 4. 다른 함수(Other Functions)  

## Socket Objects  
- socket.recvfrom(bufsize[, flags])  
    소켓에서 데이터를 수신합니다.  
    반환 값은 (bytes, address) 쌍입니다.  

## Socket timeouts  

## Example  

In [1]:
# Echo server program
import socket

HOST = ''                 # Symbolic name meaning all available interfaces
PORT = 50007              # Arbitrary non-privileged port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data: break
            conn.sendall(data)

In [ ]:
# Echo client program
import socket

HOST = 'daring.cwi.nl'    # The remote host
PORT = 50007              # The same port as used by the server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', repr(data))

In [ ]:
# Echo server program
import socket
import sys

HOST = None               # Symbolic name meaning all available interfaces
PORT = 50007              # Arbitrary non-privileged port
s = None
for res in socket.getaddrinfo(HOST, PORT, socket.AF_UNSPEC,
                              socket.SOCK_STREAM, 0, socket.AI_PASSIVE):
    af, socktype, proto, canonname, sa = res
    try:
        s = socket.socket(af, socktype, proto)
    except OSError as msg:
        s = None
        continue
    try:
        s.bind(sa)
        s.listen(1)
    except OSError as msg:
        s.close()
        s = None
        continue
    break
if s is None:
    print('could not open socket')
    sys.exit(1)
conn, addr = s.accept()
with conn:
    print('Connected by', addr)
    while True:
        data = conn.recv(1024)
        if not data: break
        conn.send(data)

In [ ]:
# Echo client program
import socket
import sys

HOST = 'daring.cwi.nl'    # The remote host
PORT = 50007              # The same port as used by the server
s = None
for res in socket.getaddrinfo(HOST, PORT, socket.AF_UNSPEC, socket.SOCK_STREAM):
    af, socktype, proto, canonname, sa = res
    try:
        s = socket.socket(af, socktype, proto)
    except OSError as msg:
        s = None
        continue
    try:
        s.connect(sa)
    except OSError as msg:
        s.close()
        s = None
        continue
    break
if s is None:
    print('could not open socket')
    sys.exit(1)
with s:
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', repr(data))

In [ ]:
import socket

# the public network interface
HOST = socket.gethostbyname(socket.gethostname())

# create a raw socket and bind it to the public interface
s = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_IP)
s.bind((HOST, 0))

# Include IP headers
s.setsockopt(socket.IPPROTO_IP, socket.IP_HDRINCL, 1)

# receive all packets
s.ioctl(socket.SIO_RCVALL, socket.RCVALL_ON)

# receive a packet
print(s.recvfrom(65565))

# disabled promiscuous mode
s.ioctl(socket.SIO_RCVALL, socket.RCVALL_OFF)

In [ ]:
socket.socket(socket.AF_CAN, socket.SOCK_DGRAM, socket.CAN_BCM)

In [ ]:
import socket
import struct


# CAN frame packing/unpacking (see 'struct can_frame' in <linux/can.h>)

can_frame_fmt = "=IB3x8s"
can_frame_size = struct.calcsize(can_frame_fmt)

def build_can_frame(can_id, data):
    can_dlc = len(data)
    data = data.ljust(8, b'\x00')
    return struct.pack(can_frame_fmt, can_id, can_dlc, data)

def dissect_can_frame(frame):
    can_id, can_dlc, data = struct.unpack(can_frame_fmt, frame)
    return (can_id, can_dlc, data[:can_dlc])


# create a raw socket and bind it to the 'vcan0' interface
s = socket.socket(socket.AF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
s.bind(('vcan0',))

while True:
    cf, addr = s.recvfrom(can_frame_size)

    print('Received: can_id=%x, can_dlc=%x, data=%s' % dissect_can_frame(cf))

    try:
        s.send(cf)
    except OSError:
        print('Error sending CAN frame')

    try:
        s.send(build_can_frame(0x01, b'\x01\x02\x03'))
    except OSError:
        print('Error sending CAN frame')


In [ ]:
import socket
import struct

# 소캣 설정
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
# sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
# sock.bind(
#     ""
#     , ""
# )
mreq = struct.pack(
    "4sl"
    , socket.inet_aton("", socket.INADDR_ANY)
)
sock.setsockopt(
    socket.IPPROTO_IP
    , socket.IP_ADD_MEMBERSHIP
    , mreq
)

while True:
    try:
        data, addr = sock.recvfrom("")
        rst = data[:-1].decode("cp949")
    except Exception as e:
        print(e)
        